# Tutorial 2: Working With Datasets
데이터는 machine learning의 중심입니다. 이 튜토리얼에서는 DeepChem이 데이터를 저장하고 관리하는 데 사용하는 데이터 세트 클래스를 소개합니다. 단순하지만 강력한 도구를 제공하여 대량의 데이터를 효율적으로 작업할 수 있습니다. 또한 NumPy, Pandas, TensorFlow 및 PyTorch와 같은 다른 인기 있는 Python 프레임워크와 쉽게 상호 작용하도록 설계되었다.

In [1]:
import deepchem as dc
dc.__version__

'2.5.0'

## Anatomy of a Dataset
마지막 튜토리얼에서 우리는 분자 용해도의 Delaney 데이터 세트를 로드했다.

In [2]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

이제 교육, 검증 및 테스트 세트의 세 가지 데이터 세트 개체가 있습니다. 각 정보에는 어떤 정보가 담겨져 있는지 이 중 하나의 문자열 표현을 print함으로써 아이디어를 얻는다.

In [4]:
print(test_dataset)

<DiskDataset X.shape: (113,), y.shape: (113, 1), w.shape: (113, 1), ids: ['c1cc2ccc3cccc4ccc(c1)c2c34' 'Cc1cc(=O)[nH]c(=S)[nH]1'
 'Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 ' ...
 'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43' 'Cc1occc1C(=O)Nc2ccccc2'
 'OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O '], task_names: ['measured log solubility in mols per litre']>


거기엔 많은 정보가 있으니까, 처음부터 시작하자. 처음에는 "DiskDataset" 레이블로 시작한다. 데이터 세트는 abstract class이다. 데이터 저장 방법에 해당하는 몇 개의 하위 클래스가 있습니다.

 - DiskDataset 세트는 디스크에 저장된 데이터 세트입니다. 총 데이터 양이 컴퓨터 메모리보다 훨씬 더 많은 경우에도 데이터는 효율적으로 액세스할 수 있는 방식으로 저장됩니다.
 - NumpyDataset은 NumPy 어레이의 모든 데이터를 저장하는 인메모리 데이터 세트이다. 이 도구는 메모리에 모두 들어갈 수 있는 중소규모 데이터 세트를 조작할 때 유용한 도구이다.
 - ImageDataset 세트는 이미지 파일의 일부 또는 전체를 디스크에 저장하는 보다 전문화된 클래스입니다. 이 기능은 이미지를 입력 또는 출력으로 하는 모델로 작업할 때 유용합니다.

이제 Dataset의 내용을 살펴보겠습니다. 모든 Dataset에는 샘플 목록이 저장됩니다. 대략적으로 말하면, 표본은 single data point입니다. 이 경우 각 표본은 분자(molecule)입니다. 다른 Dataset에서는 샘플이 실험적인 검사, 셀 라인, 이미지 또는 기타 여러 가지에 해당될 수 있습니다. 모든 샘플에 대해 Dataset에는 다음 정보가 저장됩니다.

 - X라고 하는 features. 이것은 표본을 나타내기 위해 모델에 공급되어야 하는 input입니다.
 - y라고 하는 labels입니다. 이것은 모델에서 원하는 output입니다. 교육 중에 각 표본에 대한 모델의 output을 가능한 한 y에 가깝게 만들려고 합니다.
 - weight (w라고 함). 이 값을 사용하여 일부 데이터 값이 다른 값보다 더 중요하다는 것을 나타낼 수 있습니다. 이후 튜토리얼에서는 이 방법이 어떻게 유용한지에 대한 예를 살펴보겠습니다.
 - 샘플의 고유 식별자인 ID입니다. 이것은 독특하기만 하면 무엇이든 될 수 있다. 때로는 정수 인덱스일 뿐이지만, 이 데이터 세트에서 ID는 분자를 설명하는 SMILES 문자열이다.

X, y, w 첫 번째 차원의 크기는 113입니다. 이는 이 데이터 세트에 113개의 샘플이 포함되어 있음을 의미합니다.

출력에 나열된 최종 정보는 task_names입니다. 일부 데이터 세트에는 각 샘플에 대한 여러 가지 정보가 포함되어 있습니다. 예를 들어, 표본이 분자를 나타내는 경우 데이터 집합은 해당 분자에 대한 여러 다른 실험의 결과를 기록할 수 있다. 이 데이터 세트에는 "리터당 몰 단위의 로그 가용성 측정"이라는 단일 작업만 있습니다. 또한 y와 w는 형상(113, 1)을 가진다. 이러한 배열의 두 번째 차원은 일반적으로 작업 수와 일치합니다.

## Accessing Data from a Dataset
데이터 세트에 포함된 데이터에 액세스하는 방법은 여러 가지가 있습니다. 가장 간단한 방법은 X, y, w 및 ids 속성에 직접 액세스하는 것입니다. 이 각각은 해당 정보를 NumPy 배열로 반환합니다.

In [5]:
test_dataset.y

array([[-1.60114461],
       [ 0.20848251],
       [-0.01602738],
       [-2.82191713],
       [-0.52891635],
       [ 1.10168349],
       [-0.88987406],
       [-0.52649706],
       [-0.76358725],
       [-0.64020358],
       [-0.38569452],
       [-0.62568785],
       [-0.39585553],
       [-2.05306753],
       [-0.29666474],
       [-0.73213651],
       [-1.27744393],
       [ 0.0081655 ],
       [ 0.97588054],
       [-0.10796031],
       [ 0.59847167],
       [-0.60149498],
       [-0.34988907],
       [ 0.34686576],
       [ 0.62750312],
       [ 0.14848418],
       [ 0.02268122],
       [-0.85310089],
       [-2.72079091],
       [ 0.42476682],
       [ 0.01300407],
       [-2.4851523 ],
       [-2.15516147],
       [ 1.00975056],
       [ 0.82588471],
       [-0.90390593],
       [-0.91067993],
       [-0.82455329],
       [ 1.26909819],
       [-1.14825397],
       [-2.1343556 ],
       [-1.15744727],
       [-0.1045733 ],
       [ 0.53073162],
       [-1.22567118],
       [-1

이 방법은 데이터에 액세스하는 매우 쉬운 방법이지만 사용에 =주의해야 합니다. 이렇게 하려면 모든 샘플에 대한 데이터를 한 번에 메모리에 로드해야 합니다. 이와 같은 소규모 데이터셋의 경우에는 괜찮지만 대규모 데이터셋의 경우 기존 데이터셋보다 더 많은 메모리가 필요할 수 있습니다.

더 나은 접근 방식은 데이터 세트를 반복하는 것이다. 따라서 한 번에 적은 양의 데이터만 로드하고 처리한 다음 다음 비트를 로드하기 전에 메모리를 비울 수 있습니다. itersamples() 방법을 사용하여 한 번에 하나씩 표본 위에 반복할 수 있습니다.

In [6]:
for X, y, w, id in test_dataset.itersamples():
    print(y, id)

[-1.60114461] c1cc2ccc3cccc4ccc(c1)c2c34
[0.20848251] Cc1cc(=O)[nH]c(=S)[nH]1
[-0.01602738] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-2.82191713] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-0.52891635] C1=Cc2cccc3cccc1c23
[1.10168349] CC1CO1
[-0.88987406] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[-0.52649706] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[-0.76358725] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[-0.64020358] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 
[-0.38569452] COc2c1occc1cc3ccc(=O)oc23 
[-0.62568785] CN2C(=C(O)c1ccccc1S2(=O)=O)C(=O)Nc3ccccn3 
[-0.39585553] Cc3cc2nc1c(=O)[nH]c(=O)nc1n(CC(O)C(O)C(O)CO)c2cc3C
[-2.05306753] c1ccc(cc1)c2ccc(cc2)c3ccccc3
[-0.29666474] CC34CC(=O)C1C(CCC2=CC(=O)CCC12C)C3CCC4(=O) 
[-0.73213651] c1ccc2c(c1)sc3ccccc23
[-1.27744393] CC23Cc1cnoc1C=C2CCC4C3CCC5(C)C4CCC5(O)C#C
[0.0081655] OC(C(=O)c1ccccc1)c2ccccc2
[0.97588054] OCC2OC(Oc1ccccc1CO)C(O)C(O)C2O
[-0.10796031] CC3C2CCC1(C)C=CC(=O)C(=C1C2OC3=O)C
[0.59847167] O=Cc2ccc1OCOc1c2 
[-0.60149498] CC1CCCCC1NC(=O)N

대부분의 딥 러닝 모델은 여러 샘플 배치를 한꺼번에 처리할 수 있다. `iterbatches()`를 사용하여 표본 배치에 대해 반복할 수 있습니다.

In [7]:
for X, y, w, ids in test_dataset.iterbatches(batch_size=50):
    print(y.shape)

(50, 1)
(50, 1)
(13, 1)


`iterbatches()`는 모델을 훈련시킬 때 유용한 다른 특징들을 가지고 있다. 예를 들어, `iterbatches(batch_size=100, epochs=10, deterministic=False)`는 샘플로 매번 다른 랜덤 순서로 전체 데이터 세트에 대해 10번 반복됩니다.

Datasets는 텐서플로와 PyTorch에 대한 표준 인터페이스를 사용하여 데이터를 노출시킬 수도 있다. 텐서 플로우 데이터를 가져오고 `tensorflow.data.Dataset`, `make_tf_dataset()`을 호출합니다. `torch.utils.data.IterableDataset`를 가져오고 `make_pytorch_dataset()`을 호출합니다. 자세한 내용은 API 설명서를 참조하십시오.

데이터에 액세스하는 마지막 방법은 `to_dataframe()`입니다. 이렇게 하면 데이터가 pandas `Dataframe`에 복사됩니다. 이렇게 하려면 모든 데이터를 한 번에 메모리에 저장해야 하므로 소규모 데이터 세트에서만 사용해야 합니다.

In [8]:
test_dataset.to_dataframe()

,X,y,w,ids
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.601145,1.0,c1cc2ccc3cccc4ccc(c1)c2c34
1,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.208483,1.0,Cc1cc(=O)[nH]c(=S)[nH]1
2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.016027,1.0,Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4
3,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.821917,1.0,c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
4,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.528916,1.0,C1=Cc2cccc3cccc1c23
...,...,...,...,...
108,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-1.656304,1.0,ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl
109,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.743629,1.0,c1ccsc1
110,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-2.420799,1.0,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
111,<deepchem.feat.mol_graphs.ConvMol object at 0x...,-0.209570,1.0,Cc1occc1C(=O)Nc2ccccc2


## Creating Datasets
이제 데이터셋을 직접 생성할 수 있는 방법에 대해 알아보겠습니다. `NumpyDataset`은 매우 간단합니다. 데이터가 들어 있는 배열을 생성자에게 전달하기만 하면 됩니다. 랜덤 배열을 만든 다음 NumpyDataset으로 묶습니다.

In [9]:
import numpy as np

X = np.random.random((10, 5))
y = np.random.random((10, 2))
dataset = dc.data.NumpyDataset(X=X, y=y)
print(dataset)

<NumpyDataset X.shape: (10, 5), y.shape: (10, 2), w.shape: (10, 1), ids: [0 1 2 3 4 5 6 7 8 9], task_names: [0 1]>


가중치 또는 IDs를 지정하지 않았습니다. 이것은 선택 사항입니다. `y`와 같이 말입니다. `X`만 있으면 됩니다. 이들을 빼놓았기 때문에 자동으로 `w`와 `ids` 배열을 구축해 모든 가중치를 1로 설정하고 ID를 정수 인덱스로 설정했다.

In [10]:
dataset.to_dataframe()

,X1,X2,X3,X4,X5,y1,y2,w,ids
0,0.413094,0.809116,0.607723,0.785524,0.623789,0.635325,0.952144,1.0,0
1,0.415044,0.883566,0.859434,0.329587,0.605601,0.620495,0.824312,1.0,1
2,0.704841,0.646403,0.073398,0.908802,0.260514,0.356650,0.499532,1.0,2
3,0.859875,0.823013,0.585016,0.495547,0.756869,0.030574,0.200386,1.0,3
4,0.572402,0.372774,0.255739,0.438933,0.831882,0.981404,0.818497,1.0,4
5,0.601524,0.755445,0.792169,0.429135,0.027666,0.573542,0.774427,1.0,5
6,0.559299,0.799626,0.254432,0.803145,0.468295,0.657980,0.372701,1.0,6
7,0.976519,0.605302,0.581725,0.121950,0.297080,0.814913,0.040002,1.0,7
8,0.734716,0.959906,0.315455,0.864308,0.170122,0.609677,0.820951,1.0,8
9,0.420712,0.415157,0.564970,0.141573,0.641591,0.291885,0.305796,1.0,9


`DiskDataset`를 생성하는 것은 어떻습니까? NumPy 어레이에 데이터가 있는 경우 디스크에 저장하기 위해 `DiskDataset.from_numpy()` 호출할 수 있습니다. 튜토리얼이기 때문에 임시 디렉토리에 저장하겠습니다.

In [11]:
import tempfile

with tempfile.TemporaryDirectory() as data_dir:
    disk_dataset = dc.data.DiskDataset.from_numpy(X=X, y=y, data_dir=data_dir)
    print(disk_dataset)

<DiskDataset X.shape: (10, 5), y.shape: (10, 2), w.shape: (10, 1), ids: [0 1 2 3 4 5 6 7 8 9], task_names: [0 1]>


메모리에 저장할 수 없는 대규모 데이터 세트는 어떨까요? 디스크에 수억 개의 분자에 대한 데이터가 들어 있는 대용량 파일이 있다면 어떨까요? 디스크 데이터 집합을 생성하는 프로세스가 약간 더 많이 관련됩니다. 다행히 DeepChem의 `DataLoader` 프레임워크는 대부분의 작업을 자동화할 수 있습니다. 그것은 더 큰 주제이므로, 우리는 나중에 튜토리얼로 돌아갈 것이다.